tutorial based on <a href="https://github.com/serengil/tensorflow-101/blob/master/python/emotion-analysis-from-video.py">link</a><br>

In [1]:
import warnings
import os
import time
warnings.filterwarnings("ignore")
# -1 jeżeli ma być CPU, 0 dla GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import cv2
from keras.preprocessing import image
import tensorflow as tf

Using TensorFlow backend.


In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
face_cascade = cv2.CascadeClassifier('Z:\Anaconda\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

In [5]:
from keras.models import model_from_json
model = model_from_json(open('./TransferLearning/facial_expression_model_structure.json', 'r').read())
model.load_weights('./TransferLearning/facial_expression_model_weights.h5')

In [6]:
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [7]:
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

        detected_face = img[int(y):int(y+h), int(x):int(x+w)]
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)
        detected_face = cv2.resize(detected_face, (48, 48))

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 
        predictions = model.predict(img_pixels)
        max_index = np.argmax(predictions[0])

        overlay = img.copy()
        opacity = 0.4
        cv2.rectangle(img,(x+w+10,y-25),(x+w+150,y+135),(64,64,64),cv2.FILLED)
        cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)
        cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255,255,255),1)
        cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255,255,255),1)

        emotion = ""
        cnt = 0
        for i in range(len(predictions[0])):
            if predictions[0][i]*100>0:
                cnt+=1
                emotion = "%s %s%s" % (emotions[i], round(predictions[0][i]*100, 1), '%')
                if i != max_index:
                    color = (253,0,0)
                else:
                    color = (0,0,253)
                cv2.putText(img, emotion, (int(x+w+15), int(y-12+cnt*20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    cv2.imshow('img',img)
    
    time.sleep(0.2)    

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()